In [1]:
%matplotlib widget
from bmcs_shell.folding.wb_model import WBModel
from bmcs_shell.folding.vmats2D_elastic import MATS2DElastic
import bmcs_utils.api as bu

In [2]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus=4
n_x_plus=4
wbm = WBModel(F=-1000, h=10, tmodel=tmodel)
data = dict(alpha=1.24, a=500, a_high=2000, b=900, b_high=2000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
wbm.wb_mesh.trait_set(**data);
bu.InteractiveWindow([wbm.wb_mesh.wb_cell,wbm.wb_mesh,wbm]).interact()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [3]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus, n_x_plus = 4, 4
wbm = WBModel(F=-10000, h=10, tmodel=tmodel)
data = dict(alpha=0.9, a=500, a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
wbm.wb_mesh.trait_set(**data);
wbm.get_max_vals()

2.7594219924496195

In [10]:
wbm.wb_mesh.interact()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [58]:
alpha_range = np.linspace(0.5, 1.3, 10)
a_range = np.linspace(300,500,10)

In [68]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
import numpy as np
U_grid = []
sig_grid = []
for alpha in alpha_range:
    U_list = []
    sig_list = []
    for a in a_range:
        wbm = WBModel(F=-20000, h=10, tmodel=tmodel)
        data = dict(alpha=alpha, a=a, a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
                    n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
        wbm.wb_mesh.trait_set(**data)
        max_U = wbm.get_max_vals()
        
        U1 = wbm.hist.U_t[-1]
        eps1 = wbm.fe_domain[0].xdomain.map_U_to_field(U1)
        sig1, _ = wbm.fe_domain[0].tmodel.get_corr_pred(eps1,1)
        sig_x, sig_y, tau = sig1.T
        sig_ab = np.einsum('ab...->...ab', np.array([[sig_x, tau], [tau, sig_y]], dtype=np.float))
        sig_val, sig_dir = np.linalg.eig(sig_ab)
        sig_list.append([np.min(sig_val), np.max(sig_val)])

        print(alpha, a, max_U)
        U_list.append(max_U)
    U_grid.append(U_list)
    sig_grid.append(sig_list)

0.5 300.0 5.342896740384074
0.5 322.22222222222223 5.518069993960762
0.5 344.44444444444446 5.724634399202543
0.5 366.66666666666663 5.968875797489937
0.5 388.8888888888889 6.258809029692093
0.5 411.1111111111111 6.605200510225173
0.5 433.3333333333333 7.0225519796318085
0.5 455.55555555555554 7.530667406965082
0.5 477.77777777777777 8.157553490576323
0.5 500.0 8.943808732267044
0.5888888888888889 300.0 4.058580605328572
0.5888888888888889 322.22222222222223 4.197277548851557
0.5888888888888889 344.44444444444446 4.362516333483624
0.5888888888888889 366.66666666666663 4.559840166776195
0.5888888888888889 388.8888888888889 4.796492063983594
0.5888888888888889 411.1111111111111 5.0822407407693815
0.5888888888888889 433.3333333333333 5.430591835672413
0.5888888888888889 455.55555555555554 5.860312378414578
0.5888888888888889 477.77777777777777 6.398691486347742
0.5888888888888889 500.0 7.086098601371131
0.6777777777777778 300.0 3.2578066791882847
0.6777777777777778 322.22222222222223 3.37

In [84]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(alpha_range, a_range)
Z = np.array(U_grid)
surf = ax.plot_surface(X, Y, Z, 
                       color='green',
                       #linewidth=0, antialiased=False)
                    alpha=.0, linewidth=5,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = np.array(sig_grid)[:,:,1]
# surf = ax.plot_surface(X, Y, Z,color='red', 
#                        #cmap=cm.copper,
#                     alpha=.3, linewidth=3,
#                        #cmap=cm.gist_earth,antialiased=False, 
#                 shade=False,rstride=1, cstride=1)
# Z = -np.array(sig_grid)[:,:,0]
# surf = ax.plot_surface(X, Y, Z,color='blue', 
#                        #cmap=cm.copper,
#                     alpha=.6, linewidth=3,
#                        #cmap=cm.gist_earth,antialiased=False, 
#                 shade=False,rstride=1, cstride=1)
ax.set_xlabel(r'$\alpha~[\mathrm{rad}]$')
ax.set_ylabel(r'$a~[\mathrm{mm}]$')
ax.set_zlabel(r'max compression [MPa]')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'max compression [MPa]')

In [67]:
i_alpha, i_a = 0, -1
wbm = WBModel(F=-1000, tmodel=tmodel)
print('alpha', alpha_range[i_alpha],'a', a_range[i_a] )
data = dict(alpha=alpha_range[i_alpha], 
            a=a_range[i_a], a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
wbm.wb_mesh.trait_set(**data)
wbm.wb_mesh.interact()

alpha 0.5 a 500.0


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [ ]:
15000 / 39000 